Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/CmdStan/jeIFm.ji for CmdStan [593b3428-ca2f-500c-ae53-031589ec8ddd]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/StanMCMCChain/HWPbY.ji for StanMCMCChain [8f1571ae-b3a1-52af-8ab1-32258739efdb]
└ @ Base loading.jl:1187


CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_01_model = "
data{
    int N;
    int pulled_left[N];
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    pulled_left ~ binomial( 1 , inv_logit(a) );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_10_01_model",
monitors = ["a"],
model=m_10_01_model, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_10_01_model.stan will be updated.



Input data for cmdstan

In [6]:
m_10_01_data = Dict("N" => size(df, 1),
"pulled_left" => df[:pulled_left]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_01_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
  mean   sd  5.5% 94.5% n_eff Rhat
a 0.32 0.09 0.18  0.46   166    1
";

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
     Mean        SD       Naive SE       MCSE      ESS
a 0.32551363 0.08758512 0.0013848424 0.0020994947 1000

Quantiles:
     2.5%      25.0%     50.0%      75.0%      97.5%  
a 0.1547516 0.26673375 0.3246275 0.38661525 0.49632408



End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*